# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,acapulco de juarez,16.8634,-99.8901,297.99,87,91,1.01,MX,1723517866
1,1,aykhal,66.0000,111.5000,276.65,96,100,8.55,RU,1723517968
2,2,grytviken,-54.2811,-36.5092,264.23,62,100,3.92,GS,1723517969
3,3,port lincoln,-34.7333,135.8667,289.54,66,100,4.19,AU,1723517970
4,4,tolanaro,-25.0319,46.9987,291.19,88,0,3.09,MG,1723517972


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:

# # Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
    frame_width = 700,
    frame_height = 500,
    size = 'Humidity',
    scale = 0.85,
    color = 'City'
)

# # Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_df = city_data_df.loc[(city_data_df['Humidity'] <= 30)& (city_data_df['Max Temp'] <= 300)] 

# Drop any rows with null values
ideal_df = ideal_df.dropna()

# Display sample data
ideal_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
107,107,queenstown,-31.8976,26.8753,280.90,29,96,3.17,ZA,1723518104
109,109,menongue,-14.6585,17.6910,284.27,26,0,1.00,AO,1723518107
175,175,murray bridge,-35.1167,139.2667,296.58,30,18,5.10,AU,1723518182
176,176,alice springs,-23.7000,133.8833,295.90,25,0,2.57,AU,1723518183
191,191,camacupa,-12.0167,17.4833,286.58,26,2,2.07,AO,1723518200
222,222,calama,-22.4667,-68.9333,280.42,7,0,7.72,CL,1723518005
283,283,kuruman,-27.4524,23.4325,281.66,23,95,2.14,ZA,1723518309
308,308,guiratinga,-16.3494,-53.7583,289.68,17,25,1.47,BR,1723518341
335,335,broken hill,-31.9500,141.4333,293.33,27,0,3.14,AU,1723518371
458,458,tshabong,-26.0500,22.4500,284.40,28,16,3.11,BW,1723518516


### Step 3: Create a new DataFrame called `hotel_df`.

In [10]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
107,queenstown,ZA,-31.8976,26.8753,29,
109,menongue,AO,-14.6585,17.6910,26,
175,murray bridge,AU,-35.1167,139.2667,30,
176,alice springs,AU,-23.7000,133.8833,25,
191,camacupa,AO,-12.0167,17.4833,26,
222,calama,CL,-22.4667,-68.9333,7,
283,kuruman,ZA,-27.4524,23.4325,23,
308,guiratinga,BR,-16.3494,-53.7583,17,
335,broken hill,AU,-31.9500,141.4333,27,
458,tshabong,BW,-26.0500,22.4500,28,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [16]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'categories' : "accommodation.hotel",
    'apiKey' : geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lng = row["Lng"]
    lat = row["Lat"]
    

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lng},{lat},{radius}'
    params["bias"] = f'proximity:{lng},{lat}'

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params = params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
queenstown - nearest hotel: Queens Hotel
menongue - nearest hotel: No hotel found
murray bridge - nearest hotel: Bridgeport
alice springs - nearest hotel: Aurora Alice Springs
camacupa - nearest hotel: No hotel found
calama - nearest hotel: Apart Hotel Geotel Calama
kuruman - nearest hotel: Palermo Inn
guiratinga - nearest hotel: No hotel found
broken hill - nearest hotel: Ibis Styles
tshabong - nearest hotel: No hotel found
luziania - nearest hotel: No hotel found
tafi viejo - nearest hotel: Casa Calchaqui
ondjiva - nearest hotel: Combuesa
uberaba - nearest hotel: Hotel Metrópole
chilecito - nearest hotel: Hermosina Hotel
letlhakeng - nearest hotel: No hotel found
esmeraldas - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
107,queenstown,ZA,-31.8976,26.8753,29,Queens Hotel
109,menongue,AO,-14.6585,17.6910,26,No hotel found
175,murray bridge,AU,-35.1167,139.2667,30,Bridgeport
176,alice springs,AU,-23.7000,133.8833,25,Aurora Alice Springs
191,camacupa,AO,-12.0167,17.4833,26,No hotel found
222,calama,CL,-22.4667,-68.9333,7,Apart Hotel Geotel Calama
283,kuruman,ZA,-27.4524,23.4325,23,Palermo Inn
308,guiratinga,BR,-16.3494,-53.7583,17,No hotel found
335,broken hill,AU,-31.9500,141.4333,27,Ibis Styles
458,tshabong,BW,-26.0500,22.4500,28,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [17]:
%%capture --no-display

# Configure the map plot
map_plot2 = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    hover_cols = ['Hotel Name', 'Country'],
    geo = True,
    tiles = 'OSM',
    frame_width = 700,
    frame_height = 500,
    size = 'Humidity',
    scale = 0.85,
    color = 'City'
)

# Display the map
map_plot2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)